In [1]:
from tqdm import tqdm
import gc
import time
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import numpy as np
import sys
import warnings
warnings.filterwarnings('ignore')
sys.path.append("../utils/")
from utils.MLUtils import TrainerBase, cv_trainer, getSubsetIdx
from MyModels import GrindingPredictor
from MyDataset import MemoryDataset, get_dataset, get_collate_fn
from MyDataset import sampling_rate_ae,sampling_rate_vib, project_dir, project_name, data_dir, dataDir_ae, dataDir_vib, alphabet, allowed_input_types,logical_threads,physical_threads

class Trainer(TrainerBase):
    def _forward(self, batch):
        # Example implementation:
        inputs = {k: v.to(self.device) if isinstance(v, torch.Tensor) else v for k, v in batch.items()}
        labels = batch['label'].to(self.device)
        
        outputs = self.model(inputs)
        
        return outputs, labels

class TrainingConfig:
    _instance = None

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
            cls._instance._initialize()
        return cls._instance

    def _initialize(self):
        self.model = GrindingPredictor
        self.input_type = 'all'
        self.dataset_mode = 'classical'
        self.batch_size = 4
        self.num_workers = 4
        self.learning_rate = 0.001
        self.num_epochs = 10
        self.gpu = 'cpu'
        self.repeat = 1
        self.epochs = 2
        self.folds = 10
        self.model_name = 'SmartGrinding'
        self.threads = 8

args = TrainingConfig()
device = torch.device(args.gpu if torch.cuda.is_available() else "cpu")

In [3]:
cpus = [32, 16, 2, 1]
percentage=[0.6, 0.8, 0.90, 1]
dataset = get_dataset(input_type=args.input_type, dataset_mode=args.dataset_mode, cpus = cpus, percentage=percentage)
collate_fn = get_collate_fn(input_type=args.input_type)
model_name = args.model_name

Required components: {'all'}


In [11]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
data = dataset.loaded_data

In [17]:
scaler = MinMaxScaler(feature_range=(0, 1))
data['label']
_d = scaler.fit_transform(data['label'].reshape(-1, 1)).squeeze()

In [18]:
_d

array([0.1481246 , 0.14876033, 0.12587413, 0.14049587, 0.13795296,
       0.11125238, 0.13477432, 0.12587413, 0.11824539, 0.13159568,
       0.13922441, 0.11633821, 0.12587413, 0.09980928, 0.09599491,
       0.08264463, 0.09218055, 0.09090909, 0.11125238, 0.09218055,
       0.10616656, 0.08264463, 0.07692308, 0.07565162, 0.07374444,
       0.08900191, 0.07628735, 0.07120153, 0.06675143, 0.07692308,
       0.05340114, 0.06230134, 0.06547997, 0.04577241, 0.0546726 ,
       0.06738716, 0.05721551, 0.07374444, 0.08900191, 0.08328036,
       0.12460267, 0.12205976, 0.10743802, 0.12015257, 0.10743802,
       0.11188811, 0.09663064, 0.10425938, 0.08518754, 0.09281627,
       0.0775588 , 0.04767959, 0.07183725, 0.06484425, 0.04259377,
       0.03051494, 0.03623649, 0.04831532, 0.02797203, 0.04958678,
       0.1684679 , 0.15956771, 0.12714558, 0.1411316 , 0.13095995,
       0.10425938, 0.09090909, 0.09472346, 0.10235219, 0.08328036,
       0.07883026, 0.06675143, 0.04958678, 0.08073744, 0.06993

In [3]:
loader = DataLoader(dataset, batch_size=3, shuffle=False, num_workers=0, collate_fn=collate_fn)
model = GrindingPredictor(input_type=args.input_type, interp=False)
model.to(device)

GrindingPredictor(
  (ae_spec_processor): SpectrogramProcessor(
    (conv): Sequential(
      (0): Conv2d(2, 8, kernel_size=(3, 3), stride=(2, 2))
      (1): ReLU()
      (2): AdaptiveAvgPool2d(output_size=(4, 4))
      (3): Flatten(start_dim=1, end_dim=-1)
      (4): Linear(in_features=128, out_features=32, bias=True)
    )
  )
  (vib_spec_processor): SpectrogramProcessor(
    (conv): Sequential(
      (0): Conv2d(3, 8, kernel_size=(3, 3), stride=(2, 2))
      (1): ReLU()
      (2): AdaptiveAvgPool2d(output_size=(4, 4))
      (3): Flatten(start_dim=1, end_dim=-1)
      (4): Linear(in_features=128, out_features=32, bias=True)
    )
  )
  (ae_interpreter): FeatureInterpreter(
    (attention): Sequential(
      (0): Linear(in_features=36, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=36, bias=True)
      (3): Softmax(dim=-1)
    )
    (feature_processor): Linear(in_features=36, out_features=64, bias=True)
  )
  (vib_interpreter): FeatureInte

In [4]:
batch = next(iter(loader))
inputs = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in batch.items()}
labels = batch['label'].to(device)

In [5]:
for input_type in allowed_input_types:
    print(f"Testing input type: {input_type}")
    model = GrindingPredictor(input_type=input_type, interp=False)
    model.to(device)
    outs = model.forward(inputs)
    print(f"Output shape for {input_type}: {outs.shape}")

Testing input type: ae_spec
Output shape for ae_spec: torch.Size([3, 1])
Testing input type: vib_spec
Output shape for vib_spec: torch.Size([3, 1])
Testing input type: ae_features
Output shape for ae_features: torch.Size([3, 1])
Testing input type: vib_features
Output shape for vib_features: torch.Size([3, 1])
Testing input type: ae_spec+ae_features
Output shape for ae_spec+ae_features: torch.Size([3, 1])
Testing input type: vib_spec+vib_features
Output shape for vib_spec+vib_features: torch.Size([3, 1])
Testing input type: ae_spec+ae_features+vib_spec+vib_features
Output shape for ae_spec+ae_features+vib_spec+vib_features: torch.Size([3, 1])
Testing input type: ae_features+pp
Output shape for ae_features+pp: torch.Size([3, 1])
Testing input type: vib_features+pp
Output shape for vib_features+pp: torch.Size([3, 1])
Testing input type: pp
Output shape for pp: torch.Size([3, 1])
Testing input type: all
Output shape for all: torch.Size([3, 1])
